In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import numpy as np
import pandas as pd
import os
import librosa
import cv2

from pathlib import Path
from random import shuffle
from diffsynth import ModelManager, WanVideoPipeline, save_video, VideoData

/home/xabieririzar/DiffSynth-Studio/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.path.exists("/home/xabieririzar/DiffSynth-Studio/dataset/train/gCE5175IkoY_0.mp4")

In [3]:
vid_path = Path("dataset/train")
metadata = pd.read_csv("dataset/metadata.csv")
masks = Path("dataset/lip_masks")

In [4]:
# Load latents
latents_paths = [video_path for video_path in vid_path.iterdir() if video_path.suffix == ".pth"]
# shuffle(latents_paths)

In [10]:
# C, F, H, W
latent = torch.load(latents_paths[0], map_location='cpu')['latents'].unsqueeze(0)

In [13]:
latent.shape

torch.Size([1, 16, 15, 60, 104])

In [ ]:
model_manager = ModelManager(device="cpu")
model_manager.load_models(
    [
        "models/Wan-AI/Wan2.1-T2V-1.3B/diffusion_pytorch_model.safetensors",
        "models/Wan-AI/Wan2.1-T2V-1.3B/models_t5_umt5-xxl-enc-bf16.pth",
        "models/Wan-AI/Wan2.1-T2V-1.3B/Wan2.1_VAE.pth",
    ],
    torch_dtype=torch.bfloat16, # You can set `torch_dtype=torch.float8_e4m3fn` to enable FP8 quantization.
)
pipe = WanVideoPipeline.from_model_manager(model_manager, torch_dtype=torch.bfloat16, device="cuda")
pipe.enable_vram_management(num_persistent_param_in_dit=None)

In [ ]:
# Load 5 latents
latents = []
audio_list = []

for latent_path in latents_paths[12:14]:
    latents.append(torch.load(latent_path)["latents"])
    name = latent_path.name.replace(".tensors.pth", "")

    caption = metadata[metadata["file_name"] == name]["text"].values[0]
    # Load audio
    audio_path = f"dataset/audio/{name.replace('.mp4', '.wav')}"

    print(os.path.exists(audio_path))

    audio_data, sr = librosa.load(audio_path, sr=None)
    
    # Add audio to list
    
    audio_list.append({
        'name': name,
        'audio': audio_data,
        'caption': caption
    })
    
latents = torch.stack(latents).to(pipe.device)

In [ ]:
# play audio
print(audio_list[0]['caption'])

In [ ]:
# Play the audio
from IPython.display import Audio
display(Audio(audio_list[1]['audio'], rate=41000))

In [ ]:
pipe.vae.to(pipe.device)

In [30]:
out = pipe.vae.decode(latents, device='cuda')

In [ ]:
for i in range(out.shape[0]):
    video = out[i].permute(1, 2, 3, 0).cpu().float()
    save_video(video, f"video_{i}.mp4", fps=30)